**Описание проекта**

Заказчиком проекта является сервис, который предоставляет возможность отслеживать результаты спортсменов по  фигурному катанию на различных соревнованиях. Это может помочь тренерам выстроить грамотный план тренировок и соревнований для будущих чемпионов.
Необходимо создать решение по автоматическому опеределению от какой спортивной школы выступает спортсмен, основываясь на названии введенном в ручном порядке. Одна и та же школа может быть записана по-разному.
Необходимо построить модель, которая сможет сверять введенное название с его эталонным написанием.

**План**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
- Подготовить обучающий набор данных на основе эталонного датасета
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе
- Проанализировать результат и предложить варианты улучшения


# Загрузка библиотек и данных

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.5 MB/s eta 0:00:00


In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.1 MB/s eta 0:00:00


In [4]:
!pip install accelerate -U

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [6]:
import pandas as pd
import numpy as np
import random
import torch
from sklearn.model_selection import train_test_split
import warnings
import re
import nlpaug.augmenter as naw
from datasets import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, InputExample, losses, SentencesDataset
from sentence_transformers.util import semantic_search

In [8]:
reference = pd.read_csv('/content/Школы.csv')

In [9]:
sample = pd.read_csv('/content/Примерное написание.csv')

In [10]:
reference.sample(15)

,school_id,name,region
157,158,Снеговик,Краснодарский край
216,217,СШОР № 6,Рязанская область
238,239,ФФКК РК,Республика Коми
175,176,СШ по ЗВС,Волгоградская область
246,247,Центр развития спорта,Москва
75,76,Контакт,Ямало-Ненецкий АО
232,233,ФОК городец,Нижегородская область
6,7,Айсберг,Рязанская область
64,65,Ижсталь,Удмуртская Республика
110,111,Невский лед,Санкт-Петербург


In [11]:
sample.sample(15)

,school_id,name
45,267,"ЯНАО, СШ Ямал"
248,203,"Оренбургская область, г.Оренбург, ГАУ ""СШОР №1"""
387,164,АНФСО СКФК Спартак
893,2,"ЯНАО, СШ ""Авангард"""
691,62,СПб ГБУ СШОР Звездный лед
54,264,"Свердловская область, МБУ СШ ""Юность"""
78,257,"СПб ГБУ ""ЦФКСиЗ Фрунзенского р-на"""
648,82,"Москва, МАФКК школа ""Легенда"""
820,28,"Москва, Сам обл, МАФКК ""Вдохновение"", СШОР №1"
655,81,"Сахалинская обл., ОГАУ СШ ""Кристалл"", г.Ю-С"


In [12]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


In [13]:
reference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


**Вывод:** Данных для построения модели очень мало. Таблица с эталонными написаниями содержит только 305 записей. Таблица с размеченными произвольными написаниями названий школ содержит лишь 895 записей.

Приняли решение оставить таблицу с образцами для финального тестирования модели, а для обучения и отладки модели использовать таблицу с эталонными названиями.

# Обработка таблицы с эталонными названиями школ

Создадим новую таблицу для анализа данных и создания новых признаков

In [14]:
features = reference

Проверим регионы в таблице


In [15]:
sorted(features['region'].unique())

['Алтайский край',
 'Архангельская область',
 'Астраханская область',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Волгоградская область',
 'Вологодская область',
 'Воронежская область',
 'Забайкальский край',
 'Ивановская область',
 'Иркутская область',
 'Калининградская область',
 'Калужская область',
 'Кемеровская область',
 'Кировская область',
 'Костромская бласть',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Курганская область',
 'Курская область',
 'Ленинградская область',
 'Липецкая область',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Набережные челны',
 'Нижегородская область',
 'Новгородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская область',
 'Орловская область',
 'Пензенская область',
 'Пермский край',
 'Приморский край',
 'Псковская область',
 'Республика Башкортостан',
 'Республика Карелия',
 'Республика Коми',
 'Республика Корелия',
 'Республика Крым',
 'Республика Марий Эл

При анализе списка регионов, нашли ошибки в написании некоторых названий, что привело к появлению неявных дублей. Например - Карелия/Корелия

Так же есть регионы, которые написаны несколькими способами - Республика Чувашия/Чувашская республика.

Так же имеются в списке регионов города

Приняли решение заменить названия на эталонные.

In [16]:
replacement_dict = {
    'Чувашская республика' : 'Чувашская Республика',
    'Республика Чувашия' : 'Чувашская Республика',
    'Республика Корелия' : 'Республика Карелия',
    'Удмуртская республика' : 'Удмуртская Республика',
    'Саранск' : 'Республика Мордовия',
    'Северодвинск ' : 'Архангельская область',
    'Костромская бласть' : 'Костромская область',
    'Набережные челны' : 'Республика Татарстан',
    'Ямало-Ненецкий АО' : 'Ямало-Ненецкий автономный округ',
    'ХМАО-Югра' : 'Ханты-Мансийский автономный округ - Югра',
    'Республика Саха' : 'Республика Саха'
    }

In [17]:
features['region'] = features['region'].replace(replacement_dict)

In [18]:
sorted(features['region'].unique())

['Алтайский край',
 'Архангельская область',
 'Астраханская область',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Волгоградская область',
 'Вологодская область',
 'Воронежская область',
 'Забайкальский край',
 'Ивановская область',
 'Иркутская область',
 'Калининградская область',
 'Калужская область',
 'Кемеровская область',
 'Кировская область',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Курганская область',
 'Курская область',
 'Ленинградская область',
 'Липецкая область',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Нижегородская область',
 'Новгородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская область',
 'Орловская область',
 'Пензенская область',
 'Пермский край',
 'Приморский край',
 'Псковская область',
 'Республика Башкортостан',
 'Республика Карелия',
 'Республика Коми',
 'Республика Крым',
 'Республика Марий Эл',
 'Республика Мордовия',
 'Республика Саха',
 'Республика Татарст

In [19]:
features['region'].nunique()

66

После очистки названий в списке осталось 66 регионов

Приняли решение удалить из названий кавычки и некоторые формы организации, такие как ООО, ГБУ, МБУ, ГАУ, РФ, ДО, ФАУ, которые не несут смысловой нагрузки, но усложняют анализ данных.

In [20]:
# Функция для очистки текста от лишних символов
def clean_text(text):
    #список аббевиатур для удаления
    abbreviations = ['ГАУ', 'МБУ', 'ГБУ', 'РФ', 'ГУ', 'ГБОУ', 'ДО', 'ФАУ', 'ООО', 'ИП']
    #регулярка для их удаления
    pattern = r'\b(?:' + '|'.join(abbreviations) + r')\b'
    text = re.sub(pattern, '', text).strip()
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\«\»\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # возвращаем очищенные данные
    return text

In [21]:
# создаем список для хранения очищенных данных
cleaned_text = []
# для каждого названия из столбца name
for name in features['name']:
    # очищаем данные
    name = clean_text(name)
    # добавляем очищенные данные в список cleaned_text
    cleaned_text.append(name)
# записываем очищенные данные в новую колонку 'cleaned_text'
features['name'] = cleaned_text

Приняли решение просмотреть список школ, внутри регионов, для того чтобы исключить возможность появления дополнительных дублей

In [22]:
features[['name','region']].duplicated().sum()

6

In [23]:
features[features[['name','region']].duplicated()]

,school_id,name,region
11,12,Академия ФКК,Республика Мордовия
39,40,Голубева,Костромская область
67,68,Керриган,Республика Карелия
186,187,СШ 6,Мурманская область
213,214,СШОР 4,Чувашская Республика
286,289,Пируэт,Санкт-Петербург


В таблице нашлись 6 полных дубликата записей, отличающихся только id номером. Приняли решение их удалить

In [24]:
ind_to_remove = [11, 39, 67, 186, 213, 289]
features = features.drop(ind_to_remove)
features = features.reset_index(drop=True)

In [25]:
features = features.groupby('region').apply(lambda x: x.sort_values('name')).reset_index(drop=True)
pd.set_option('display.max_rows', None)
features

,school_id,name,region
0,305,Прогресс,Алтайский край
1,22,Беломорец,Архангельская область
2,9,Звездочка,Архангельская область
3,66,Каскад,Архангельская область
4,159,Созвездие,Астраханская область
5,51,ДЮСШ по ЗВС,Белгородская область
6,26,Брянск,Брянская область
7,36,СК,Брянская область
8,178,СШ по ФК,Брянская область
9,59,ДЮСШ 8,Владимирская область


Просмотрев весь список, отсортированный по регионам, мы нашли дополнительные дубли, не очевидные для машинного поиска. Приняли решение их удалить.

In [26]:
ind_to_remove = [147, 148, 164, 255, 259, 136, 205, 200, 264, 271]
features = features.drop(ind_to_remove)
features = features.reset_index(drop=True)

In [27]:
pd.reset_option('display.max_rows')

Теперь список очищен от дубликатов, которые можно обнаружить на первый взгляд и готов к дальнейшей работе.

# Создание новых признаков

Добавим в таблицу со школами сокращенные названия регионов - это данные взятые из "Правила вида спорта "стрельба из арбалета" (утв. приказом Минспорта России от 22.02.2019 N 157) раздел - СУБЪЕКТЫ РОССИЙСКОЙ ФЕДЕРАЦИИ, СОКРАЩЕННЫЕ НАЗВАНИЯ, ПРИНЯТЫЕ ДЛЯ ПРОТОКОЛОВ СОРЕВНОВАНИЙ.

Создам словарь для названия региона и его скоращения

In [28]:
abbrev = pd.read_excel('/content/сокращения регионов.xlsx')

In [29]:
abbrev.loc[79]

N                                        80
Субъект РФ                город Севастополь
Код для протокола                       СВС
Административный центр          Севастополь
Name: 79, dtype: object

In [30]:
abbrev.head(5)

,N,Субъект РФ,Код для протокола,Административный центр
0,1,Республика Адыгея,АДГ,Майкоп
1,2,Республика Алтай,АЛТ,Горно-Алтайск
2,3,Республика Башкортостан,БАШ,Уфа
3,4,Республика Бурятия,БУР,Улан-Удэ
4,5,Республика Дагестан,ДАГ,Махачкала


Удалю из названия регионов слово город

In [31]:
abbrev['Субъект РФ'] = abbrev['Субъект РФ'].str.replace('город ', '')

In [32]:
abbrev['Субъект РФ'] = abbrev['Субъект РФ'].str.replace(r'\(.*?\)', '', regex=True)

In [33]:
region_to_abbrev = dict(zip(abbrev['Субъект РФ'],abbrev['Код для протокола']))

In [34]:
features['reg_abbrev'] = features['region'].map(region_to_abbrev)

In [35]:
features.sample(3)

,school_id,name,region,reg_abbrev
135,109,МУ СШ 1,Республика Коми,КОМ
178,274,Голден Айс Петергоф СШ Манеж,Санкт-Петербург,СПБ
263,135,Прибой,Тюменская область,ТЮМ


In [36]:
features[features['reg_abbrev'].isna()]

,school_id,name,region,reg_abbrev
146,140,РОО СФФК РСЯ,Республика Саха,NaN
147,142,РССШ по ЗВС,Республика Саха,NaN


In [37]:
features['reg_abbrev'] = features['reg_abbrev'].fillna('САХ')
features[features['reg_abbrev'].isna()]

,school_id,name,region,reg_abbrev


Кроме таких признаных сокращений регионов часто используются просто первые буквы названий региона, создадим колонку с таким признаком. Республика Саха = РС

In [38]:
def extract_first_letters(text):
  words = text.split() #разбиваем название региона на слова
  first_letters = [word[0].upper() for word in words]
  return ''.join(first_letters)

In [39]:
features['first_letters'] = features['region'].apply(extract_first_letters)

In [40]:
features.sample(3)

,school_id,name,region,reg_abbrev,first_letters
52,35,ВСШОР,Ленинградская область,ЛЕН,ЛО
178,274,Голден Айс Петергоф СШ Манеж,Санкт-Петербург,СПБ,С
44,80,Красноярск,Красноярский край,КРЯ,КК


In [41]:
features[features['first_letters'].str.len() == 1].sample(3)

,school_id,name,region,reg_abbrev,first_letters
176,30,Вилеса Айс,Санкт-Петербург,СПБ,С
66,106,Москвич,Москва,МСК,М
76,1836,Триумф,Москва,МСК,М


У городов федерального значения из первых букв остается только одна - заменим это сокращение на тоже общепринятые.Для Москвы часто упротребимое сокращение Мос, для Санкт-Петербурга - СПб

In [42]:
city_codes = {
    'Москва' : 'Мос',
    'Санкт-Петербург' : 'СПб'
}

def assign_code(city, code): #функция для замены значений для Москвы и Санкт-Петербурга
  if city in city_codes:
    return city_codes[city]
  else:
    return code #оставляем исходное значение, если город другой

features['first_letters'] = features.apply(
    lambda row: assign_code(row['region'], row['first_letters']), axis=1
    )

In [43]:
features.sample(3)

,school_id,name,region,reg_abbrev,first_letters
7,36,СК,Брянская область,БРЯ,БО
86,42,Давыдов,Московская область,МСО,МО
136,218,СШОР 1,Республика Коми,КОМ,РК


Для дальнейшего сравнения создадим эталонное название составленное из Наименования школы и региона - title

Так же создадим новые признаки из сочетаний названия школы + региона в разном порядке и с сокращениями


In [44]:
# Функция для создания новых столбцов по шаблону
def create_new_column(df, new_col_name, template):
    df[new_col_name] = df.apply(lambda row: template.format(**row), axis=1)

# Шаблоны для новых столбцов
templates = {
    'title': '{name} {region}',
    'name2': '{name} {reg_abbrev}',
    'name3': '{name} {first_letters}',
    'name4': '{region} {name}',
    'name5': '{reg_abbrev} {name}',
    'name6': '{first_letters} {name}',
}

# Создание новых столбцов на основе шаблонов
for new_col_name, template in templates.items():
    create_new_column(features, new_col_name, template)

In [45]:
features.sample(10)

,school_id,name,region,reg_abbrev,first_letters,title,name2,name3,name4,name5,name6
36,100,МБФСУ СШ,Кемеровская область,КЕМ,КО,МБФСУ СШ Кемеровская область,МБФСУ СШ КЕМ,МБФСУ СШ КО,Кемеровская область МБФСУ СШ,КЕМ МБФСУ СШ,КО МБФСУ СШ
151,230,МО ЦСКА,Республика Татарстан,ТАТ,РТ,МО ЦСКА Республика Татарстан,МО ЦСКА ТАТ,МО ЦСКА РТ,Республика Татарстан МО ЦСКА,ТАТ МО ЦСКА,РТ МО ЦСКА
268,87,Ледовый дворец спорта,Ханты-Мансийский автономный округ - Югра,ХМН,ХАО-Ю,Ледовый дворец спорта Ханты-Мансийский автоном...,Ледовый дворец спорта ХМН,Ледовый дворец спорта ХАО-Ю,Ханты-Мансийский автономный округ - Югра Ледов...,ХМН Ледовый дворец спорта,ХАО-Ю Ледовый дворец спорта
263,135,Прибой,Тюменская область,ТЮМ,ТО,Прибой Тюменская область,Прибой ТЮМ,Прибой ТО,Тюменская область Прибой,ТЮМ Прибой,ТО Прибой
14,176,СШ по ЗВС,Волгоградская область,ВГГ,ВО,СШ по ЗВС Волгоградская область,СШ по ЗВС ВГГ,СШ по ЗВС ВО,Волгоградская область СШ по ЗВС,ВГГ СШ по ЗВС,ВО СШ по ЗВС
88,91,Лыткарино,Московская область,МСО,МО,Лыткарино Московская область,Лыткарино МСО,Лыткарино МО,Московская область Лыткарино,МСО Лыткарино,МО Лыткарино
249,163,Союз мастеров спорта,Ставропольский край,СТВ,СК,Союз мастеров спорта Ставропольский край,Союз мастеров спорта СТВ,Союз мастеров спорта СК,Ставропольский край Союз мастеров спорта,СТВ Союз мастеров спорта,СК Союз мастеров спорта
51,18,Арена,Курская область,КРС,КО,Арена Курская область,Арена КРС,Арена КО,Курская область Арена,КРС Арена,КО Арена
169,206,СШОР No 1,Самарская область,САМ,СО,СШОР No 1 Самарская область,СШОР No 1 САМ,СШОР No 1 СО,Самарская область СШОР No 1,САМ СШОР No 1,СО СШОР No 1
161,216,СШОР 6,Ростовская область,РСТ,РО,СШОР 6 Ростовская область,СШОР 6 РСТ,СШОР 6 РО,Ростовская область СШОР 6,РСТ СШОР 6,РО СШОР 6


In [46]:
features.shape

(290, 11)

In [47]:
features.columns

Index(['school_id', 'name', 'region', 'reg_abbrev', 'first_letters', 'title',
       'name2', 'name3', 'name4', 'name5', 'name6'],
      dtype='object')

Добавлю так же признаки с ошибкой в написании названия школы

In [48]:
# Функция для создания ошибок в строке
def introduce_typo(name):
    if len(name) > 1:
        pos = random.randint(0, len(name) - 1)
        random_char = random.choice('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ')
        name = name[:pos] + random_char + name[pos + 1:]
    return name


def create_columns_with_typos(df, columns_to_process, num_columns_with_typos=3):
    for col in columns_to_process:
        for i in range(1, num_columns_with_typos + 1):
            new_col_name = f'{col}with_typos{i}'
            df[new_col_name] = df[col].apply(lambda x: introduce_typo(x))

# Список колонок, для которых нужно создать ошибки
columns_to_process = ['title', 'name', 'name2', 'name3', 'name4', 'name5', 'name6']

# Создание новых колонок с ошибками
create_columns_with_typos(features, columns_to_process)

print(features.head(1))

   school_id      name          region reg_abbrev first_letters  \
0        305  Прогресс  Алтайский край        АЛТ            АК   

                     title         name2        name3  \
0  Прогресс Алтайский край  Прогресс АЛТ  Прогресс АК   

                     name4         name5  ... name3with_typos3  \
0  Алтайский край Прогресс  АЛТ Прогресс  ...      Орогресс АК   

          name4with_typos1         name4with_typos2         name4with_typos3  \
0  Алтайский краШ Прогресс  Алтайскиь край Прогресс  Алшайский край Прогресс   

  name5with_typos1 name5with_typos2 name5with_typos3 name6with_typos1  \
0     АЛТ Прзгресс     АЛТ Прогчесс     АЛТ Пробресс      АК ПГогресс   

  name6with_typos2 name6with_typos3  
0      АК ПрогресВ      АК Срогресс  

[1 rows x 32 columns]


In [49]:
features.columns

Index(['school_id', 'name', 'region', 'reg_abbrev', 'first_letters', 'title',
       'name2', 'name3', 'name4', 'name5', 'name6', 'titlewith_typos1',
       'titlewith_typos2', 'titlewith_typos3', 'namewith_typos1',
       'namewith_typos2', 'namewith_typos3', 'name2with_typos1',
       'name2with_typos2', 'name2with_typos3', 'name3with_typos1',
       'name3with_typos2', 'name3with_typos3', 'name4with_typos1',
       'name4with_typos2', 'name4with_typos3', 'name5with_typos1',
       'name5with_typos2', 'name5with_typos3', 'name6with_typos1',
       'name6with_typos2', 'name6with_typos3'],
      dtype='object')

In [50]:
features.shape

(290, 32)

In [51]:
features.loc[features['school_id']==162]

,school_id,name,region,reg_abbrev,first_letters,title,name2,name3,name4,name5,...,name3with_typos3,name4with_typos1,name4with_typos2,name4with_typos3,name5with_typos1,name5with_typos2,name5with_typos3,name6with_typos1,name6with_typos2,name6with_typos3
141,162,Союз мастеров спорта,Республика Крым,КРМ,РК,Союз мастеров спорта Республика Крым,Союз мастеров спорта КРМ,Союз мастеров спорта РК,Республика Крым Союз мастеров спорта,КРМ Союз мастеров спорта,...,Еоюз мастеров спорта РК,Республика Крым Союз мастеров споЙта,РеспублиЪа Крым Союз мастеров спорта,Республика Крым Союз мастеров сп рта,КРМ Яоюз мастеров спорта,КРМ Союз мастеровшспорта,КРМ СЙюз мастеров спорта,РК ССюз мастеров спорта,РК Союз мастеровЛспорта,РК Союз мастеров спорга


Создали достаточно данных, для того чтобы провести обучение модели. Удалим все строки, которые не подходят для обучения.

In [52]:
features = features.drop(columns=['region', 'reg_abbrev', 'first_letters'])

In [53]:
features.sample(3)

,school_id,name,title,name2,name3,name4,name5,name6,titlewith_typos1,titlewith_typos2,...,name3with_typos3,name4with_typos1,name4with_typos2,name4with_typos3,name5with_typos1,name5with_typos2,name5with_typos3,name6with_typos1,name6with_typos2,name6with_typos3
64,102,Мечта,Мечта Москва,Мечта МСК,Мечта Мос,Москва Мечта,МСК Мечта,Мос Мечта,Мечта МоЦква,Мечта Мясква,...,Мечта Бос,Москва Мейта,Москва Мечтш,Москва Мзчта,МСе Мечта,МСШ Мечта,МСК Мелта,Мос Мемта,Еос Мечта,Моб Мечта
6,26,Брянск,Брянск Брянская область,Брянск БРЯ,Брянск БО,Брянская область Брянск,БРЯ Брянск,БО Брянск,Брянск Бронская область,Брянск БрянЦкая область,...,Брянск ИО,Брянская область Бтянск,БЛянская область Брянск,БНянская область Брянск,БРЯхБрянск,эРЯ Брянск,БРЯ Беянск,БО Брянса,БОпБрянск,БО БрХнск
259,157,Смена,Смена Томская область,Смена ТОМ,Смена ТО,Томская область Смена,ТОМ Смена,ТО Смена,Смена ТомАкая область,СменЧ Томская область,...,СменаНТО,ТомскЖя область Смена,Томская область Смина,Томская область Смеца,ТОЕ Смена,ТГМ Смена,ТГМ Смена,ТОЧСмена,ТОыСмена,ТО мена


Преобразую широкую таблицу в вертикальную, сохранив верные идентификаторы школ.

In [54]:
schools_features_melt = features.melt(id_vars=['school_id', 'title'], var_name='name_type', value_name='value')

In [55]:
schools_features_melt = schools_features_melt.drop(columns=['name_type'])

In [56]:
schools_features_melt = schools_features_melt.sample(frac=1).reset_index(drop=True)

In [57]:
schools_features_melt.sample(5)

,school_id,title,value
5236,2,Авангард Ямало-Ненецкий автономный округ,ЯМН АванЧард
7775,57,ДЮСШ 4 Санкт-Петербург,ДЮСШ 4 Санкт-ПАтербург
3885,50,Дымка Кировская область,КИР Дымка
3616,160,Созвездие Москва,Созвефдие
1146,229,Фаворит Санкт-Петербург,Фаворвт


In [58]:
train, test = train_test_split(schools_features_melt, test_size=0.2, random_state=123)

In [59]:
train.shape

(6264, 3)

In [60]:
test.shape

(1566, 3)

# LaBSE from the box

Решила закодировать названия в эмбеддинги. Для кодирования использую модель LaBSE из sentence-transformers с huggingface. Эту модель можно использовать для отображения 109 языков в общее векторное пространство.
Создам модель и функцию для кодирования

In [61]:
model = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

Кодируем целевые названия в эмбеддинги

In [62]:
corpus = model.encode(features.title.values)

In [63]:
query = model.encode(test.value.values)

In [64]:
model_search = semantic_search(query, corpus, top_k=1)

In [65]:
test['search_id'] = [x[0]['corpus_id'] for x in model_search]

In [66]:
test['candidate_name'] = features.title.values[test.search_id.values]
test = test.drop(columns=['search_id'])

In [67]:
test.sample(10)

,school_id,title,value,candidate_name
1812,257,ЦФКСиЗ Фрунзенского района Санкт-Петербург,ЦФКСиЗ фрунзенского района СПБ,ЦФКСиЗ Фрунзенского района Санкт-Петербург
6844,54,ДЮСШ 10 Самарская область,ДЮСШ 10 Сапарская область,ДЮСШ 10 Самарская область
628,65,Ижсталь Удмуртская Республика,Ижсталь,Ижсталь Удмуртская Республика
3389,109,МУ СШ 1 Республика Коми,МУ щШ 1,УОР 1 Московская область
1115,96,МАФКК Москва,МСК МАхКК,МАФКК Москва
7122,192,СШОР Колпинского района Санкт-Петербург,СШОР Болпинского района СПБ,СШОР Колпинского района Санкт-Петербург
2139,132,Пируэт Хабаровский край,ХАБ ПиЬуэт,Пируэт Хабаровский край
649,232,ФКК Тверская область,ФКК ТВю,ФКК Тверская область
1461,223,ТИТУЛ Санкт-Петербург,ТИТУЛ СанкМ-Петербург,ТИТУЛ Санкт-Петербург
4010,285,Отделение фигурного катания Ника Клуб СБС Санк...,ОтделениФ фигурного катания Ника Клуб СБС СПб,Отделение фигурного катания Ника Клуб СБС Санк...


In [68]:
(test.title == test.candidate_name).sum()/test.shape[0]

0.814176245210728

# LaBSE training

In [69]:
# Пример позитивных пар предложений
positive_pairs = train[['title', 'value']].values

# Создание списка InputExample
train_examples1 = [InputExample(texts=[s1, s2]) for s1, s2 in positive_pairs]

# Создание датасета и DataLoader
train_dataset = SentencesDataset(train_examples1, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# Определение функции потерь GISTEmbedLoss
guide_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
train_loss = losses.GISTEmbedLoss(model=model, guide=guide_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [70]:
# Использование модели SentenceTransformer для обучения
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

Step,Training Loss
500,1.707600


Проверка дообученноймодели на тестовых данных

In [71]:
corpus = model.encode(features.title.values)

In [72]:
query = model.encode(test.value.values)

In [73]:
model_search = semantic_search(query, corpus, top_k=1)

In [74]:
len(model_search)

1566

In [75]:
test['search_id'] = [x[0]['corpus_id'] for x in model_search]

In [76]:
test['candidate_name_train'] = features.title.values[test.search_id.values]
test = test.drop(columns=['search_id'])

In [77]:
(test.title == test.candidate_name_train).sum()/test.shape[0]

0.933588761174968

In [78]:
test.sample(10)

,school_id,title,value,candidate_name,candidate_name_train
5558,203,СШОР 1 Оренбургская область,аШОР 1 ОО,УОР 1 Московская область,СШОР 1 Оренбургская область
1405,256,ЦФКСиЗ Московского района Санкт-Петербург,СПБ ЦФКСиЗ МосковскоЬо района,ЦФКСиЗ Московского района Санкт-Петербург,ЦФКСиЗ Московского района Санкт-Петербург
2699,256,ЦФКСиЗ Московского района Санкт-Петербург,ЦФИСиЗ Московского района,ЦФКСиЗ Московского района Санкт-Петербург,ЦФКСиЗ Московского района Санкт-Петербург
5588,148,Салют Московская область,СалютВМО,Салют Московская область,Салют Московская область
3180,266,Юность Чувашская Республика,Юннсть,Юность Чувашская Республика,Юность Чувашская Республика
5799,47,Динамо Ивановская область,ДКнамо ИО,Динамо Ивановская область,Динамо Ивановская область
4548,208,СШОР 1 Московского района Санкт-Петербург,Санкт-Петербург СШОРБ1 Московского района,СШОР 1 Московского района Санкт-Петербург,СШОР 1 Московского района Санкт-Петербург
6756,14,Альберта Демченко Московская область,Альберта Демченко Московская областЩ,Альберта Демченко Московская область,Альберта Демченко Московская область
2284,51,ДЮСШ по ЗВС Белгородская область,ДЮСШ по ЗВСРБЕЛ,СШОР по ЗВС Тверская область,ДЮСШ по ЗВС Белгородская область
4880,222,Темпо Санкт-Петербург,Санкт-Петербурв Темпо,Темпо Санкт-Петербург,Темпо Санкт-Петербург


In [79]:
test[test['title'] != test['candidate_name_train']]

,school_id,title,value,candidate_name,candidate_name_train
3502,165,Спартак Чувашская Республика,Жпартак,Спартак Санкт-Петербург,Спартак Санкт-Петербург
152,205,СШОР 1 Республика Башкортостан,РБ СШОд 1,СШОР 1 Республика Мордовия,СШОР 1 Республика Мордовия
5376,47,Динамо Ивановская область,Динвмо,Динамо Владимирская область,Динамо Владимирская область
2763,49,Династия Санкт-Петербург,Йинастия Санкт-Петербург,Буревестник Санкт-Петербург,Иванова Санкт-Петербург
1554,237,ФФКК Республика Карелия,ФФККУКАР,ФФКК РК Республика Коми,ФФККВО Владимирская область
...,...,...,...,...,...
6620,174,СШ ЗВС Красноярский край,пРЯ СШ ЗВС,СШ ЗВС Красноярский край,СШ по ЗВС Пензенская область
7748,265,Юность Ханты-Мансийский автономный округ - Югра,ХМН Юность,Юность Мурманская область,Юность Мурманская область
7241,183,СШ 1 Нижегородская область,СШ х,СШ 2 Республика Башкортостан,СШ по хоккею с шайбой Смоленская область
2683,181,СШ 1 Вологодская область,СШ Г ВО,СШ по ЗВС Волгоградская область,СШ по ЗВС Волгоградская область


Дообученная модель показала отличный результат на тестовых данных, опледеляет верную школу с точностью 0.93. Некоторые аугументации исказили название школы до неузнаваемости, но так же проблемы в опледелении возникают, когда школы с аналогичными названиями присутствуют в разных регионах. Так например школа Ангелы Плющенко есть как в Москве, так и в Московской области.

Проверим модель на примере данных от заказчика

# LaBSE test on references

In [80]:
school_sample = sample

In [81]:
school_sample.sample(5)

,school_id,name
719,59,"Владимирская область, ДЮСШ №8"
785,38,"ЧОУ ДО ДЮСШ ""Голден Айс"""
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
867,14,"Московская область, МУ СШОР Альберта Демченко"
136,244,"ГБУ ДО МАФК, школа Хрустальный"


In [82]:
school_sample.shape

(895, 2)

Для начала очищаем данные от символов в названиях

In [83]:
# очищеннаем данные с помощью уже готовой функции
cleaned_text = []

for name in school_sample['name']:
    name = clean_text(name)
    cleaned_text.append(name)
school_sample['title'] = cleaned_text
school_sample = school_sample.drop(columns=['name'])

In [84]:
school_sample.sample(5)

,school_id,title
115,248,Московская область Центр спортивных технологий
783,40,Костромская область КО СШОР им АВ Голубева
718,60,Иркутская обл МАУ АГО СШ Ермак
794,37,Нижегородская область НО СШОР по ЛВС
180,223,Санкт-Петербург КФК ТИТУЛ


Создадим новый корпус текстов для кодирования в эмбеддинги и дальнейшего семантического поиска, используя уже дообученную модель LaBSE

In [85]:
query_sample = model.encode(school_sample.title.values)

In [86]:
model_search_sample = semantic_search(query_sample, corpus, top_k=1)

In [87]:
school_sample['search_id'] = [x[0]['corpus_id'] for x in model_search_sample]

In [88]:
school_sample['score'] = [x[0]['score'] for x in model_search_sample]

In [89]:
school_sample['candidate_name'] = features.title.values[school_sample.search_id.values]
school_sample['candidate_id'] = features.school_id.values[school_sample.search_id.values]
school_sample = school_sample.drop(columns=['search_id'])

In [90]:
school_sample_merged = school_sample.merge(reference, left_on='school_id', right_on='school_id', how='left')

In [91]:
school_sample_merged.sample(5)

,school_id,title,score,candidate_name,candidate_id,name,region
642,83,Ростовская область АНО СК ФКК Ледовая академия,0.801062,Ледовая академия Ростовская область,83,Ледовая академия,Ростовская область
250,203,Оренбургская область СШОР 1,0.966299,СШОР 1 Оренбургская область,203,СШОР 1,Оренбургская область
59,263,Мурманская область МАУ СШ Юность,0.830974,Юность Мурманская область,263,Юность,Мурманская область
626,88,Республика Тататрстан СШ Ледокол,0.812065,Ледокол Республика Татарстан,88,Ледокол,Республика Татарстан
767,46,Владимирская область ВРО ОГО ВФСО Динамо,0.760005,Динамо Владимирская область,46,Динамо,Владимирская область


In [92]:
(school_sample.school_id == school_sample.candidate_id).sum()/school_sample.shape[0]

0.7452513966480447

На рабочем примере данных точность модели упала до 0.74, необходимо проанализировать в каких случаях модель допускает ошибки. Для большей наглядности можно присоединить датасет с эталонными названиями к таблице с образцами и вывести те строки в которых было дано ошибочное предсказание

In [93]:
mistakes = school_sample_merged[school_sample_merged['school_id'] != school_sample_merged['candidate_id']]
mistakes = mistakes.sort_values(by='score')
mistakes.head(10)

,school_id,title,score,candidate_name,candidate_id,name,region
68,259,МО Финляндский АНО ФСО Чемпион,0.521847,ФФКК ЯНАО Ямало-Ненецкий автономный округ,240,Чемпион,Санкт-Петербург
29,284,МО Коломяги Глайд,0.529509,Олимп Калужская область,118,ГЛАЙД,Санкт-Петербург
745,50,КОГАУ СШ Дымка,0.553885,ДЮСШ 2 Курганская область,55,Дымка,Кировская область
853,19,ЯНАО МАУ СШ Арктур,0.572153,СШ Ханты-Мансийский автономный округ - Югра,173,Арктур,Ямало-Ненецкий автономный округ
32,279,МО Прометей КФК Созвездие Айс,0.574985,МосСпортОбъект ЛЦ Звезда Москва,294,Созвездие Айс,Санкт-Петербург
23,293,АНО Лига фигурного катания,0.576162,Армия фигурного катания Московская область,21,АНО Лига фигурного катания,Москва
885,5,Айсдрим,0.588223,Айсберг Республика Крым,6,Ice Dream Айс Дрим,Санкт-Петербург
67,259,МО Финляндский АНО ФСО КФК Чемпион,0.599499,ФФККВО Владимирская область,241,Чемпион,Санкт-Петербург
749,48,Санкт-Петрбург СПб ГБПОУ Академия ледовых видо...,0.600333,СШОР по фигурному катанию на коньках Санкт-Пет...,198,Динамо Санкт-Петербург,Санкт-Петербург
748,49,МО 7-й округ Династия,0.606121,ДЮСШ 7 Иркутская область,58,Династия,Санкт-Петербург


In [94]:
mistakes.tail(10)

,school_id,title,score,candidate_name,candidate_id,name,region
828,26,Брянская область СК Брянск,0.929984,СК Брянская область,36,Брянск,Брянская область
830,26,Брянская область СК Брянск,0.929984,СК Брянская область,36,Брянск,Брянская область
832,26,Брянская область СК Брянск,0.929984,СК Брянская область,36,Брянск,Брянская область
833,26,СК Брянск Брянская область,0.939700,СК Брянская область,36,Брянск,Брянская область
288,197,РМ СШОР по ФКК Республика Мордовия,0.940599,СШОР по ФКК Республика Мордовия,200,СШОР по фигурному катанию на конька,Республика Мордовия
580,96,Москва Московская академия фигурного катания н...,0.944373,Московская академия фигурного катания на коньк...,107,МАФКК,Москва
582,96,Москва Московская академия фигурного катания н...,0.945418,Московская академия фигурного катания на коньк...,107,МАФКК,Москва
618,90,Свердловская обл МБОУ СШ 8 Локомотив,0.966612,МБОУ СШ 8 Локомотив Свердловская область,296,Локомотив,Свердловская область
616,90,Свердловская область МБОУ СШ 8 Локомотив,0.971928,МБОУ СШ 8 Локомотив Свердловская область,296,Локомотив,Свердловская область
298,194,Нижегородская область НО СШОР по ЛВС,0.973450,НО СШОР по ЛВС Нижегородская область,37,СШОР по ЛВС,Нижегородская область


После сортировки по уровню уверенности модели в полученном результате, можно сделать вывод, что в ситуациях, когда модель ошиблась, но показывала вероятность правильного ответа около 0.9 проблема видна либо в том, что названия очень близки, либо в эталонной таблице присутствуют скрытые дубликаты. Так например модель отностит "Свердловская область МБОУ СШ 8 Локомотив" к "МБОУ СШ 8 Локомотив Свердловская область", что возможно является одной и той же школой. Так же например "Нижегородская область НО СШОР по ЛВС" отнесена к "НО СШОР по ЛВС Нижегородская область".

Из тех школ, которым модель присвоила id с трудом, некоторые возможно имеют ошибочно размеченный лейбл. Так например школа "МО Коломяги ДЮСШ Голден айс" размечена как школа "Голден айс Санкт-Петербург".

# Анализ результата исследования

Было проведено исследование данных заказчика. Для анализа были предоставлены два небольших набора данных - с эталонными названиями школ и и с примерами их разного написания.

Так как данных для построения модели не достаточно было принято решение создать собственный датасет для обучения, используя эталонные наименования. Датасет был расширен путем добавления различных комбинаций из названия + региона, аббервиатуры региона, его сокращения, а так же путем внесения орфографических ошибок в получившееся наименование.

В результате таких комбинаций был подготовлен датасет на 7830 строк и разделен на тестовый и тренировочный.

В качестве модели были опробованы 2 модели DeepPavlov и LaBSE из sentence-transformers huggingface. DeepPavlov показала намного более долгий процесс обучения и более низкий результат в базовой версии. Поэтому была выбрана модель LaBSE. Эту модель можно использовать для отображения 109 языков в общее векторное пространство. Она очень хорошо работает так же с текстами на русском языке, но без доработки показала результат 82% точности.

Тренировочный датасет был использован для дообучения модели, после чего ее результат улучшился до 93% точности.

**Модель, использованная в исследовании**
- SentenceTransformer('sentence-transformers/LaBSE').
- Так же для поиска реультата была использована утилита из sentence_transformers - semantic_search, которая сравнивает полученные в результате кодирования эмбеддинги и находит максимально похожую пару.

**Для оценки результата** использовалась метрика Точность (Accuracy) — она измеряет долю правильных предсказаний по отношению к общему числу предсказаний. Точность показывает, насколько хорошо модель или система распознаёт или предсказывает значения.

**Инструкция по запуску модели:**

In [95]:
#очистить ваши данные от лишних символов и абберевиатур с помощью функции
#cleaned_text = []

#for name in ВАША_ТАБЛИЦА['name']:
    #name = clean_text(name)
    #cleaned_text.append(name)
#ВАША_ТАБЛИЦА['title'] = cleaned_text
#ВАША_ТАБЛИЦА = ВАША_ТАБЛИЦА.drop(columns=['name'])
#вместо колонки name появится колонка title с очищенным названием

In [96]:
#query = model.encode(ВАША_ТАБЛИЦА.title.values)
#закодировать названия в эмбеддинги с помощью предобученной модели

In [97]:
#model_search = semantic_search(query, corpus, top_k=1)
#провести поиск семантического сходства с эталонной таблицей, заранее записанной в corpus

In [98]:
#ВАША_ТАБЛИЦА['search_id'] = [x[0]['corpus_id'] for x in model_search]
#ВАША_ТАБЛИЦА['candidate_name'] = features.title.values[ВАША_ТАБЛИЦА.search_id.values]
#ВАША_ТАБЛИЦА = ВАША_ТАБЛИЦА.drop(columns=['search_id'])
#добавляем в таблицу эталонное название, для наглядности

**Рекомендации заказчику**

Для дальнейшего улучшения работы модели необходимо доработать датасет с эталонными названиями, исключить дубли. Так же было бы полезно присвоить каждой школе какой-то уникальный код, например ИНН, который в дальнейшем могли бы указывать представители школ прирегистрации на соревнование. Это создаст задел на будущее, который упростит идентификацию бюджетных учреждений, так как именно они имеют самые сложные буквенные наименования.